# Segmenting and Clustering Neighborhoods in Toronto

### 1. Start by creating a new Notebook for this assignment.

##### Preprocessing

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

In [2]:
# getting data from internet
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page= requests.get(url).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(wikipedia_page,'xml')
#print(soup.prettify())

### 3. To create the above dataframe:

##### 3.a) Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. <br> & <br> 3.b) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [3]:
table = soup.find('table')
Postcode      = []
Borough       = []
Neighbourhood = []

#print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    counter = 1
    Postcode_var      = '-1'
    Borough_var       = '-1'
    Neighbourhood_var = '-1'
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Borough_var == 'Not assigned'):
            continue
            
    try:
        if (tag_a_Borough is None):
            continue
        else:
            if (tag_a_Neighbourhood is None):
                    Neighbourhood_var = Borough_var
    except:
        pass
    
    if(Postcode_var == '-1' or Borough_var == '-1' or Neighbourhood_var == '-1'):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

##### 3.c) More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [4]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in unique_p:
    p_var = '';
    b_var = '';
    n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 100


##### 3.d) The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [5]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
#df_toronto.loc[df_toronto['Postcode'] == 'M5V']
#df_toronto.loc[df_toronto['Postcode'] == 'M9A']
df_toronto

,Postcode,Borough,Neighbourhood
0,M6P,West Toronto,"High Park, West Toronto"
1,M1J,Scarborough,Scarborough Village
2,M4H,East York,Thorncliffe Park
3,M5E,Downtown Toronto,Berczy Park
4,M8Z,Etobicoke,"Etobicoke, Mimico NW, The Queensway West, Etob..."
5,M5X,Downtown Toronto,"First Canadian Place, Underground city"
6,M4G,East York,Leaside
7,M4E,East Toronto,The Beaches
8,M1H,Scarborough,Scarborough
9,M2J,North York,"North York, Henry Farm, North York"


##### 3.e) In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [6]:
df_toronto.shape

(100, 3)

### 4. Submit a link to your Notebook on your Github repository

### 5.  In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood

##### Since it is not able to obtain coordinates from using geocoder.google, the given csv file is used

In [7]:
# Since it is not able to obtain coordinates from using geocoder.google, the given csv file is used
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2020-01-24 11:55:18--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 158.85.108.83, 158.85.108.86, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.83|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2020-01-24 11:55:18--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-24 11:55:19--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-24 11:55:20--  https:/

In [8]:
df_cord = pd.read_csv('GeoCord.csv') # Read the csv data
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# Create Latitude and Longitude columns in df_assigned
df_toronto['Latitude'] = np.nan
df_toronto['Longitude'] = np.nan

# For each postcode in df_assigned, find corresponding coordinates in df_cord and assign it to df_assigned
for idx in df_toronto.index:
    cord_idx = df_cord['Postal Code'] == df_toronto.loc[idx, 'Postcode']
    df_toronto.at[idx, 'Latitude'] = df_cord.loc[cord_idx, 'Latitude'].values
    df_toronto.at[idx, 'Longitude'] = df_cord.loc[cord_idx, 'Longitude'].values

In [10]:
# Display the results
df_toronto.head(20)
#df_toronto.loc[df_toronto['Postcode'] == 'M5G']
#df_toronto.loc[df_toronto['Postcode'] == 'M2H']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6P,West Toronto,"High Park, West Toronto",43.661608,-79.464763
1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
2,M4H,East York,Thorncliffe Park,43.705369,-79.349372
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M8Z,Etobicoke,"Etobicoke, Mimico NW, The Queensway West, Etob...",43.628841,-79.520999
5,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
6,M4G,East York,Leaside,43.709060,-79.363452
7,M4E,East Toronto,The Beaches,43.676357,-79.293031
8,M1H,Scarborough,Scarborough,43.773136,-79.239476
9,M2J,North York,"North York, Henry Farm, North York",43.778517,-79.346556


### 6.- Explore and cluster the neighborhoods in Toronto

In [11]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto